In [46]:
from langchain_community.document_loaders import PyPDFLoader
files=["Java_Collections.pdf","Git.pdf","CPP_OOPS.pdf","Data_science.pdf",
       "ML Interview Ques.pdf","Software Testing Interview Questions.pdf","Deep_Learning.pdf",
       "Cracking The Machine Learning Interview.pdf"]
data=[]
for i in files:
    loader=PyPDFLoader("./books/"+i)
    data+=loader.load()

In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
docs=text_splitter.split_documents(data)

print("Total number of documents:",len(docs))

Total number of documents: 1708


In [48]:
from langchain_chroma import Chroma 
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

embeddings =GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [50]:
persist_dir = "/Users/rishabhralli/Documents/JIIT/Major1/implementation/supernewvecstore"

vectorstore=Chroma.from_documents(documents=docs,embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),persist_directory=persist_dir)

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.3,max_tokens=10000)

In [23]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt =(
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context. If the entered topic does not qualify as a skill or"
    "even if it is a skill and does not pertain to the information in the document, then say that you cannot answer."
    "else you may generate five questions and their answers using the following format, first a question is generated then followed by its answer."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [27]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":20})

In [28]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [31]:
res=rag_chain.invoke({"input":"What is Python"})

In [32]:
print(res["answer"])

Based on the provided text, Python is a highly developed yet simple and effective programming language.  It extensively supports machine learning applications, acts as a connecting link between various software, tools, and datasets, and creates high-quality reproducible analysis that is flexible and powerful.  It also provides a robust package ecosystem for diverse needs and is useful when solving data-oriented problems.  Additionally, it enables the programmer to create functions without a name and almost instantly.

